In [6]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)



Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (2000000 rows)...
Finished inserting 2000000 rows into 'products' in 36.64s.


So a covering index should be a index that contains all the values needed for the query. And idea is that it should be faster since you dont have to traverse the clustered index to get all the data you need

So for starters i want to test if i select two fields but use only a only a index with a single key that i search for in the db. This should be slower than if it were a covering index even though the where is only on one of the keys

In [7]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 111336 rows fetched, 202.82 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size,idx_size,403,const,223618,100.0,None


In [8]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size_and_name ON products(size, name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT name, size FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 111336 rows fetched, 188.50 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_size_and_name,idx_size_and_name,403,const,206652,100.0,Using index


It seems to be faster. I did have to up the records size to 2m in order to see the difference.

I want to try with also using some sql methods in a covering index

In [9]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1 rows fetched, 217.82 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,category_id,category_id,5,const,123776,100.0,None


In [11]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_category_id ON products(category_id);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1 rows fetched, 6.24 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,"idx_category_id_and_price,idx_category_id",idx_category_id_and_price,5,const,119948,100.0,Using index


In [10]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_category_id_and_price ON products(category_id, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT AVG(price) FROM products WHERE category_id = 1;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1 rows fetched, 7.60 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_category_id_and_price,idx_category_id_and_price,5,const,119948,100.0,Using index


Here we can see in the extra that it is using the index which just means that it gets all the information it needs from the index and dont have to traverse the clustered index to look at the data. Pretty cool